In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import torch

from iit.model_pairs.strict_iit_model_pair import StrictIITModelPair

from cases.paren_checker import HighLevelParensBalanceChecker, test_HL_parens_balancer_components, BalancedParensDataset
from cases.left_greater import HighLevelLeftGreater, test_HL_left_greater_components, LeftGreaterDataset
from cases.duplicate_remover import HighLevelDuplicateRemover, test_HL_duplicate_remover_components, DuplicateRemoverDataset
from cases.unique_extractor import HighLevelUniqueExtractor, test_HL_unique_extractor_components, UniqueExtractorDataset
from poly_hl_model import PolyHLModel, PolyModelDataset

In [ ]:
n_epochs = 300
n_samples = 1_000
#iit_weight = 1. / siit_weight = 0.4 / behavior_weight = 1. works!
training_args = {
    "batch_size": 256,
    "num_workers": 0,
    "use_single_loss": True,
    "behavior_weight": 0.5, #basically doubles the strict weight's job.
    "iit_weight": 1.,
    "strict_weight": 0.5,
    "clip_grad_norm": 1.0,
    "iit_weight_schedule" : lambda s, i: s,
    "strict_weight_schedule" : lambda s, i: s,
    "behavior_weight_schedule" : lambda s, i: s, #0.955*s if 0.955**i > 0.01 else s, #have behavior weight decay over time
    "early_stop" : True,
    "lr_scheduler": torch.optim.lr_scheduler.LinearLR,
    "scheduler_kwargs": dict(start_factor=1, end_factor=0.3, total_iters=int(n_epochs)),
    "optimizer_kwargs": dict(lr=1e-3, weight_decay=1e-4, betas=(0.9, 0.9)), #1e-8 was 237 epochs
    "scheduler_val_metric": ["val/accuracy", "val/IIA"], #for ReduceLRonPlateau
    "scheduler_mode": "max", #for ReduceLRonPlateau
    "siit_sampling" : "sample_all",
    "seed" : 42
}

: 

: 

: 

: 

# Paren Checker

In [ ]:
test_HL_parens_balancer_components()

All Balance tests passed!


True

: 

: 

: 

In [ ]:
hl_model = HighLevelParensBalanceChecker()
corr = hl_model.get_correspondence()
dataset = BalancedParensDataset(N_samples=n_samples, n_ctx=hl_model.get_ll_model_cfg().n_ctx, seed=42)
train_set, test_set = dataset.get_IIT_train_test_set()

making IIT dataset


: 

: 

: 

In [ ]:
i = 6
print(dataset.get_dataset()[i]['tokens'], dataset.get_dataset()[i]['labels'])

[0, 2, 2, 2, 3, 3, 3, 2, 2, 3, 3, 2, 3, 2, 3] [[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0]]


: 

: 

: 

In [ ]:
print(dataset.get_dataset().shape)
print(dataset.get_dataset()[:10]['tokens'])
print(dataset.get_dataset()[:10]['labels'])
for i in range(10):
    tokens, labels, hl_outputs = dataset.get_dataset()[i]['tokens'], dataset.get_dataset()[i]['labels'], hl_model((torch.tensor(dataset.get_dataset()[i]['tokens'])[None,:], None, None))
    nonzero = (torch.tensor(labels) - hl_outputs[0].cpu()).nonzero()
    if nonzero.numel() > 0:
        print(tokens, torch.unique(nonzero[:,0]))
        bad_indices = torch.unique(nonzero[:,0]).tolist()
        for idx in bad_indices:
            print(labels[idx], hl_outputs[0,idx])

(1000, 4)
[[0, 2, 2, 3, 3, 2, 2, 3, 2, 3, 2, 2, 3, 3, 3], [0, 2, 2, 2, 3, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2], [0, 3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3], [0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3], [0, 3, 3, 2, 3, 3, 3, 3, 3, 2, 2, 3, 2, 2, 2], [0, 2, 3, 2, 2, 2, 2, 3, 3, 3, 2, 2, 3, 2, 3], [0, 2, 2, 2, 3, 3, 3, 2, 2, 3, 3, 2, 3, 2, 3], [0, 3, 2, 2, 2, 2, 2, 3, 2, 3, 3, 2, 3, 2, 3], [0, 2, 2, 3, 2, 3, 2, 3, 3, 2, 2, 2, 3, 3, 3], [0, 3, 3, 2, 3, 2, 3, 2, 3, 3, 2, 2, 3, 2, 2]]
[[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0]], [[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0

: 

: 

: 

In [ ]:

ll_model = hl_model.get_ll_model(seed=42)
model_pair = StrictIITModelPair(hl_model=hl_model, ll_model=ll_model, corr=corr, training_args=training_args)

model_pair.train(
    train_set=train_set,
    test_set=test_set,
    optimizer_cls=torch.optim.AdamW,
    epochs=n_epochs,
)

training_args={'batch_size': 256, 'num_workers': 0, 'early_stop': True, 'lr_scheduler': <class 'torch.optim.lr_scheduler.LinearLR'>, 'scheduler_val_metric': ['val/accuracy', 'val/IIA'], 'scheduler_mode': 'max', 'scheduler_kwargs': {'start_factor': 1, 'end_factor': 0.1, 'total_iters': 300}, 'optimizer_kwargs': {'lr': 0.001, 'weight_decay': 1e-08, 'betas': (0.9, 0.9)}, 'clip_grad_norm': 1.0, 'seed': 42, 'detach_while_caching': True, 'lr': 0.001, 'atol': 0.05, 'use_single_loss': True, 'iit_weight': 1.0, 'behavior_weight': 1.0, 'strict_weight': 1.0, 'siit_sampling': 'sample_all', 'iit_weight_schedule': <function <lambda> at 0x17525b740>, 'strict_weight_schedule': <function <lambda> at 0x17525b600>, 'behavior_weight_schedule': <function <lambda> at 0x17525b380>}


Training Epochs:   0%|          | 0/300 [00:00<?, ?it/s]

Training Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch 1: lr: 9.97e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 1.00e+00, train/iit_loss: 9.39e-01, train/behavior_loss: 9.21e-01, train/strict_loss: 9.22e-01, val/iit_loss: 7.37e-01, val/IIA: 87.60, val/accuracy: 88.38, val/strict_accuracy: 88.38
Epoch 2: lr: 9.94e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 1.00e+00, train/iit_loss: 6.94e-01, train/behavior_loss: 6.92e-01, train/strict_loss: 6.92e-01, val/iit_loss: 6.33e-01, val/IIA: 94.27, val/accuracy: 95.04, val/strict_accuracy: 95.04
Epoch 3: lr: 9.91e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 1.00e+00, train/iit_loss: 6.03e-01, train/behavior_loss: 5.98e-01, train/strict_loss: 5.99e-01, val/iit_loss: 5.73e-01, val/IIA: 94.27, val/accuracy: 95.04, val/strict_accuracy: 95.04
Epoch 4: lr: 9.88e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 1.00e+00, train/iit_loss: 5.71e-01, train/behavior_loss: 5.50e-01, train/strict_loss: 5.50e-01, val/

: 

: 

: 

In [ ]:
i = 6
input = torch.tensor(dataset.get_dataset()[i]['tokens'])
print(input)
print(torch.round(torch.nn.functional.softmax(model_pair.ll_model.forward(input), dim=-1)))
print(hl_model((torch.tensor(input)[None,:], None, None))) 
print(dataset.get_dataset()[i]['labels'])

tensor([0, 2, 2, 2, 3, 3, 3, 2, 2, 3, 3, 2, 3, 2, 3])
tensor([[[0., 0., 1., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.]]], device='mps:0', grad_fn=<RoundBackward0>)
tensor([[[0., 0., 1., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.]]], device='mps:0')
[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0

/var/folders/ry/qny1f95136l2lpppg_d78n6c0000gq/T/ipykernel_12645/775366424.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(hl_model((torch.tensor(input)[None,:], None, None)))


: 

: 

: 

# Left > Right

In [ ]:
test_HL_left_greater_components()

All left greater tests passed!


True

: 

: 

: 

In [ ]:
hl_model = HighLevelLeftGreater()
corr = hl_model.get_correspondence()
dataset = LeftGreaterDataset(N_samples=n_samples, n_ctx=hl_model.get_ll_model_cfg().n_ctx, seed=42)
train_set, test_set = dataset.get_IIT_train_test_set()
print(hl_model.hook_dict)
print(list(corr.keys()))

making IIT dataset
{'input_hook': HookPoint(), 'paren_counts_hook': HookPoint(), 'mlp0_hook': HookPoint()}
[input_hook, paren_counts_hook, mlp0_hook]


: 

: 

: 

In [ ]:
ll_model = hl_model.get_ll_model(seed=42)

model_pair = StrictIITModelPair(hl_model=hl_model, ll_model=ll_model, corr=corr, training_args=training_args)
model_pair.train(
    train_set=train_set,
    test_set=test_set,
    optimizer_cls=torch.optim.AdamW,
    epochs=n_epochs,
)

training_args={'batch_size': 256, 'num_workers': 0, 'early_stop': True, 'lr_scheduler': <class 'torch.optim.lr_scheduler.LinearLR'>, 'scheduler_val_metric': ['val/accuracy', 'val/IIA'], 'scheduler_mode': 'max', 'scheduler_kwargs': {'start_factor': 1, 'end_factor': 0.1, 'total_iters': 300}, 'optimizer_kwargs': {'lr': 0.001, 'weight_decay': 1e-08, 'betas': (0.9, 0.9)}, 'clip_grad_norm': 1.0, 'seed': 42, 'detach_while_caching': True, 'lr': 0.001, 'atol': 0.05, 'use_single_loss': True, 'iit_weight': 1.0, 'behavior_weight': 1.0, 'strict_weight': 1.0, 'siit_sampling': 'sample_all', 'iit_weight_schedule': <function <lambda> at 0x31fa5a020>, 'strict_weight_schedule': <function <lambda> at 0x3348df380>, 'behavior_weight_schedule': <function <lambda> at 0x3536019e0>}


Epoch 1: lr: 9.97e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.34e+00, train/behavior_loss: 1.31e+00, train/strict_loss: 1.31e+00, val/iit_loss: 1.25e+00, val/IIA: 47.82, val/accuracy: 58.19, val/strict_accuracy: 57.12
Epoch 2: lr: 9.94e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.21e+00, train/behavior_loss: 1.17e+00, train/strict_loss: 1.18e+00, val/iit_loss: 1.14e+00, val/IIA: 59.29, val/accuracy: 67.27, val/strict_accuracy: 66.39
Epoch 3: lr: 9.91e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.13e+00, train/behavior_loss: 1.09e+00, train/strict_loss: 1.10e+00, val/iit_loss: 1.08e+00, val/IIA: 63.77, val/accuracy: 67.94, val/strict_accuracy: 66.85
Epoch 4: lr: 9.88e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.10e+00, train/behavior_loss: 1.05e+00, train/strict_loss: 1.06e+00, val/

: 

: 

: 

# Duplicate remover
case 19 in circuits-bench

In [ ]:

test_HL_duplicate_remover_components()

tensor([[0, 2, 1, 3, 4, 2, 3, 1, 1],
        [0, 2, 3, 4, 1, 1, 1, 1, 1],
        [0, 2, 3, 4, 1, 1, 1, 1, 1]])
All DuplicateRemover tests passed!


True

: 

: 

: 

In [ ]:
hl_model = HighLevelDuplicateRemover()
corr = hl_model.get_correspondence()
dataset = DuplicateRemoverDataset(N_samples=n_samples, n_ctx=hl_model.get_ll_model_cfg().n_ctx, seed=42)
train_set, test_set = dataset.get_IIT_train_test_set()
print(hl_model.hook_dict)
print(list(corr.keys()))

tensor([[0, 2, 1, 1, 1, 1, 1, 4, 3, 2, 4, 3, 1, 4, 1]])
tensor([[0, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 3, 2]])
tensor([[0, 2, 1, 1, 1, 1, 3, 4, 1, 2, 4, 1, 2, 4, 3]])
tensor([[0, 2, 1, 1, 1, 1, 3, 4, 1, 3, 2, 1, 4, 3, 1]])
tensor([[0, 2, 1, 1, 1, 1, 4, 2, 1, 4, 1, 2, 4, 3, 2]])
tensor([[0, 2, 1, 1, 1, 1, 4, 3, 1, 2, 1, 3, 4, 2, 1]])
tensor([[0, 2, 1, 1, 1, 1, 4, 1, 2, 4, 3, 2, 4, 3, 2]])
tensor([[0, 2, 1, 1, 1, 1, 4, 1, 1, 3, 4, 2, 3, 4, 1]])
tensor([[0, 2, 1, 1, 1, 3, 2, 3, 2, 1, 4, 3, 1, 1, 2]])
tensor([[0, 2, 1, 1, 1, 3, 2, 3, 1, 2, 4, 3, 4, 3, 2]])
tensor([[0, 2, 1, 1, 1, 3, 2, 3, 4, 3, 2, 4, 3, 1, 1]])
tensor([[0, 2, 1, 1, 1, 3, 1, 2, 3, 4, 3, 4, 2, 3, 2]])
tensor([[0, 2, 1, 1, 1, 3, 1, 1, 1, 4, 3, 4, 3, 1, 2]])
tensor([[0, 2, 1, 1, 1, 3, 1, 1, 1, 4, 3, 4, 1, 1, 2]])
tensor([[0, 2, 1, 1, 1, 3, 1, 1, 4, 2, 1, 3, 4, 2, 1]])
tensor([[0, 2, 1, 1, 1, 3, 4, 3, 4, 1, 1, 3, 4, 1, 2]])
tensor([[0, 2, 1, 1, 1, 4, 2, 3, 1, 2, 4, 3, 1, 4, 1]])
tensor([[0, 2, 1, 1, 1, 4, 3, 2, 3, 1, 4, 3, 4, 

: 

: 

: 

In [ ]:
ll_model = hl_model.get_ll_model(seed=42)

model_pair = StrictIITModelPair(hl_model=hl_model, ll_model=ll_model, corr=corr, training_args=training_args)
model_pair.train(
    train_set=train_set,
    test_set=test_set,
    optimizer_cls=torch.optim.AdamW,
    epochs=n_epochs,
)

training_args={'batch_size': 256, 'num_workers': 0, 'early_stop': True, 'lr_scheduler': <class 'torch.optim.lr_scheduler.LinearLR'>, 'scheduler_val_metric': ['val/accuracy', 'val/IIA'], 'scheduler_mode': 'max', 'scheduler_kwargs': {'start_factor': 1, 'end_factor': 0.1, 'total_iters': 300}, 'optimizer_kwargs': {'lr': 0.001, 'weight_decay': 1e-08, 'betas': (0.9, 0.9)}, 'clip_grad_norm': 1.0, 'seed': 42, 'detach_while_caching': True, 'lr': 0.001, 'atol': 0.05, 'use_single_loss': True, 'iit_weight': 1.0, 'behavior_weight': 1.0, 'strict_weight': 1.0, 'siit_sampling': 'sample_all', 'iit_weight_schedule': <function <lambda> at 0x31fa5a020>, 'strict_weight_schedule': <function <lambda> at 0x3348df380>, 'behavior_weight_schedule': <function <lambda> at 0x3536019e0>}


tensor([[0, 3, 2,  ..., 1, 4, 1],
        [0, 2, 3,  ..., 4, 1, 1],
        [0, 2, 4,  ..., 4, 3, 1],
        ...,
        [0, 2, 3,  ..., 4, 2, 3],
        [0, 2, 1,  ..., 1, 1, 2],
        [0, 2, 4,  ..., 2, 1, 4]])
tensor([[0, 3, 2,  ..., 1, 4, 1],
        [0, 2, 3,  ..., 4, 1, 1],
        [0, 2, 4,  ..., 4, 3, 1],
        ...,
        [0, 2, 3,  ..., 4, 2, 3],
        [0, 2, 1,  ..., 1, 1, 2],
        [0, 2, 4,  ..., 2, 1, 4]])
tensor([[0, 3, 1,  ..., 3, 2, 3],
        [0, 2, 3,  ..., 4, 3, 2],
        [0, 2, 3,  ..., 1, 1, 3],
        ...,
        [0, 2, 3,  ..., 2, 4, 3],
        [0, 2, 4,  ..., 4, 3, 1],
        [0, 2, 4,  ..., 1, 4, 2]])
tensor([[0, 3, 1,  ..., 3, 2, 3],
        [0, 2, 3,  ..., 4, 3, 2],
        [0, 2, 3,  ..., 1, 1, 3],
        ...,
        [0, 2, 3,  ..., 2, 4, 3],
        [0, 2, 4,  ..., 4, 3, 1],
        [0, 2, 4,  ..., 1, 4, 2]])
tensor([[0, 3, 2,  ..., 3, 4, 1],
        [0, 2, 3,  ..., 1, 1, 3],
        [0, 2, 3,  ..., 4, 2, 4],
        ...,
        [0, 2

: 

: 

: 

# Unique Extractor

In [ ]:
test_HL_unique_extractor_components()

All UniqueExtractor tests passed!


True

: 

: 

: 

In [ ]:
hl_model = HighLevelUniqueExtractor()
corr = hl_model.get_correspondence()
dataset = UniqueExtractorDataset(N_samples=n_samples, n_ctx=hl_model.get_ll_model_cfg().n_ctx, seed=42)
train_set, test_set = dataset.get_IIT_train_test_set()
print(hl_model.hook_dict)
print(list(corr.keys()))

making IIT dataset
{'input_hook': HookPoint(), 'counter_head': HookPoint(), 'appeared_mlp': HookPoint(), 'mask_mlp': HookPoint(), 'output_mlp': HookPoint()}
[input_hook, counter_head, appeared_mlp, mask_mlp, output_mlp]


: 

: 

: 

In [ ]:
ll_model = hl_model.get_ll_model(seed=42)

model_pair = StrictIITModelPair(hl_model=hl_model, ll_model=ll_model, corr=corr, training_args=training_args)
model_pair.train(
    train_set=train_set,
    test_set=test_set,
    optimizer_cls=torch.optim.AdamW,
    epochs=n_epochs,
)

training_args={'batch_size': 256, 'num_workers': 0, 'early_stop': True, 'lr_scheduler': <class 'torch.optim.lr_scheduler.LinearLR'>, 'scheduler_val_metric': ['val/accuracy', 'val/IIA'], 'scheduler_mode': 'max', 'scheduler_kwargs': {'start_factor': 1, 'end_factor': 0.1, 'total_iters': 300}, 'optimizer_kwargs': {'lr': 0.001, 'weight_decay': 1e-08, 'betas': (0.9, 0.9)}, 'clip_grad_norm': 1.0, 'seed': 42, 'detach_while_caching': True, 'lr': 0.001, 'atol': 0.05, 'use_single_loss': True, 'iit_weight': 1.0, 'behavior_weight': 1.0, 'strict_weight': 1.0, 'siit_sampling': 'sample_all', 'iit_weight_schedule': <function <lambda> at 0x31fa5a020>, 'strict_weight_schedule': <function <lambda> at 0x3348df380>, 'behavior_weight_schedule': <function <lambda> at 0x3536019e0>}


Epoch 1: lr: 9.97e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.20e+00, train/behavior_loss: 1.19e+00, train/strict_loss: 1.19e+00, val/iit_loss: 9.68e-01, val/IIA: 77.45, val/accuracy: 80.13, val/strict_accuracy: 80.13
Epoch 2: lr: 9.94e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 1.00e+00, train/iit_loss: 9.05e-01, train/behavior_loss: 8.93e-01, train/strict_loss: 8.93e-01, val/iit_loss: 8.47e-01, val/IIA: 81.88, val/accuracy: 84.56, val/strict_accuracy: 84.56
Epoch 3: lr: 9.91e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 1.00e+00, train/iit_loss: 8.42e-01, train/behavior_loss: 7.83e-01, train/strict_loss: 7.84e-01, val/iit_loss: 7.78e-01, val/IIA: 83.73, val/accuracy: 88.41, val/strict_accuracy: 88.13
Epoch 4: lr: 9.88e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 1.00e+00, train/iit_loss: 7.23e-01, train/behavior_loss: 6.98e-01, train/strict_loss: 6.99e-01, val/

: 

: 

: 

: 

: 

: 

# Poly Model

In [ ]:
# n_epochs = 1000
# #iit_weight = 1. / siit_weight = 0.4 / behavior_weight = 1. works!
# training_args = {
#     "batch_size": 256,
#     "num_workers": 0,
#     "use_single_loss": True,
#     "behavior_weight": 0.5, #basically doubles the strict weight's job.
#     "iit_weight": 1.,
#     "strict_weight": 0.5,
#     "clip_grad_norm": 1.0,
#     "iit_weight_schedule" : lambda s, i: s,
#     "strict_weight_schedule" : lambda s, i: s,
#     "behavior_weight_schedule" : lambda s, i: s, #0.955*s if 0.955**i > 0.01 else s, #have behavior weight decay over time
#     "early_stop" : True,
#     "lr_scheduler": torch.optim.lr_scheduler.LinearLR,
#     "scheduler_kwargs": dict(start_factor=1, end_factor=0.01, total_iters=int(0.8*n_epochs)),
#     "optimizer_kwargs": dict(lr=1e-3, weight_decay=1e-4, betas=(0.9, 0.7)), #betas=(0.9, 0.7) trains better than (0.9, 0.9)
#     "scheduler_val_metric": ["val/accuracy", "val/IIA"], #for ReduceLRonPlateau
#     "scheduler_mode": "max", #for ReduceLRonPlateau
#     "siit_sampling" : "sample_all"
# }

: 

: 

: 

## LeftGreater + ParensBalancer

In [ ]:
cases = [HighLevelLeftGreater, HighLevelParensBalanceChecker]
poly_hl_model = PolyHLModel(hl_classes=cases, size_expansion=2)
corr = poly_hl_model.get_correspondence()

print(n_samples)
dataset1 = LeftGreaterDataset(N_samples=n_samples, n_ctx=15, seed=42)
dataset2 = BalancedParensDataset(N_samples=n_samples, n_ctx=15, seed=42)
dsets = [dataset1,dataset2]
poly_dataset = PolyModelDataset(dsets, n_ctx=poly_hl_model.cfg.n_ctx)


1000


/Users/evananders/far_cluster/circuits-benchmark-saes/poly_bench/poly_hl_model.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens = torch.cat([torch.tensor(dataset.tokens) for dataset in datasets], dim=0)
/Users/evananders/far_cluster/circuits-benchmark-saes/poly_bench/cases/utils.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.inputs = t.tensor(inputs).to(int)


: 

: 

: 

In [ ]:
ll_model = poly_hl_model.get_ll_model(seed=42)
corr = poly_hl_model.get_correspondence()
# for k, v in corr.items():
#     print(k, v)

# for k, v in poly_hl_model.corr_mapping.items():
#     print(k, v)

train_set, test_set = poly_dataset.get_IIT_train_test_set()
model_pair = StrictIITModelPair(hl_model=poly_hl_model, ll_model=ll_model, corr=corr, training_args=training_args)
print(model_pair.nodes_not_in_circuit)
model_pair.train(
    train_set=train_set,
    test_set=test_set,
    optimizer_cls=torch.optim.AdamW,
    epochs=n_epochs,
)

[LLNode(name='blocks.0.attn.hook_z', index=[:, :, 1, :], subspace=None), LLNode(name='blocks.0.attn.hook_z', index=[:, :, 2, :], subspace=None), LLNode(name='blocks.0.attn.hook_z', index=[:, :, 3, :], subspace=None), LLNode(name='blocks.0.mlp.hook_post', index=[:], subspace=None), LLNode(name='blocks.1.attn.hook_z', index=[:, :, 0, :], subspace=None), LLNode(name='blocks.1.attn.hook_z', index=[:, :, 2, :], subspace=None), LLNode(name='blocks.2.attn.hook_z', index=[:, :, 0, :], subspace=None), LLNode(name='blocks.2.attn.hook_z', index=[:, :, 1, :], subspace=None), LLNode(name='blocks.2.attn.hook_z', index=[:, :, 2, :], subspace=None), LLNode(name='blocks.2.attn.hook_z', index=[:, :, 3, :], subspace=None), LLNode(name='blocks.3.attn.hook_z', index=[:, :, 0, :], subspace=None), LLNode(name='blocks.3.attn.hook_z', index=[:, :, 1, :], subspace=None), LLNode(name='blocks.3.attn.hook_z', index=[:, :, 2, :], subspace=None)]
training_args={'batch_size': 256, 'num_workers': 0, 'early_stop': True

Training Epochs:   0%|          | 0/300 [00:00<?, ?it/s]

Training Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Epoch 1: lr: 9.97e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 1.00e+00, train/iit_loss: 8.27e-01, train/behavior_loss: 7.32e-01, train/strict_loss: 7.96e-01, val/iit_loss: 5.37e-01, val/IIA: 69.51, val/accuracy: 83.05, val/strict_accuracy: 82.01
Epoch 2: lr: 9.94e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 1.00e+00, train/iit_loss: 5.05e-01, train/behavior_loss: 2.73e-01, train/strict_loss: 4.66e-01, val/iit_loss: 3.96e-01, val/IIA: 80.59, val/accuracy: 97.33, val/strict_accuracy: 94.20
Epoch 3: lr: 9.91e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 1.00e+00, train/iit_loss: 3.76e-01, train/behavior_loss: 1.35e-01, train/strict_loss: 5.70e-01, val/iit_loss: 3.02e-01, val/IIA: 83.12, val/accuracy: 98.60, val/strict_accuracy: 96.07
Epoch 4: lr: 9.88e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 1.00e+00, train/iit_loss: 3.59e-01, train/behavior_loss: 1.24e-01, train/strict_loss: 5.28e-01, val/

KeyboardInterrupt: 

: 

: 

: 

### DuplicateRemover + LeftGreater

In [ ]:
cases = [HighLevelDuplicateRemover, HighLevelLeftGreater]
poly_hl_model = PolyHLModel(hl_classes=cases, size_expansion=2)
corr = poly_hl_model.get_correspondence()
for k, v in corr.items():
    print(k, v)

print()
for k, v in poly_hl_model.corr_mapping.items():
    print(k, v)

input_hook {LLNode(name='blocks.0.hook_resid_pre', index=[:], subspace=None)}
mlp_hooks.0 {LLNode(name='blocks.0.mlp.hook_post', index=[:], subspace=None)}
attn_hooks.0.0 {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 0, :], subspace=None)}
attn_hooks.0.1 {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 1, :], subspace=None)}
task_hook {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 2, :], subspace=None)}
mlp_hooks.1 {LLNode(name='blocks.1.mlp.hook_post', index=[:], subspace=None)}

blocks.0.mlp.hook_post [[prev_equal_hook], [mlp0_hook]]
blocks.0.attn.hook_z.0 [[prev_token_hook], None]
blocks.0.attn.hook_z.1 [None, [paren_counts_hook]]
blocks.0.attn.hook_z.2 [None, None]
blocks.0.attn.hook_z.3 [None, None]
blocks.1.mlp.hook_post [[output_hook], None]
blocks.1.attn.hook_z.0 [None, None]
blocks.1.attn.hook_z.1 [None, None]
blocks.1.attn.hook_z.2 [None, None]
blocks.1.attn.hook_z.3 [None, None]


: 

: 

: 

In [ ]:
dataset1 = DuplicateRemoverDataset(N_samples=n_samples, n_ctx=15, seed=42)
dataset2 = LeftGreaterDataset(N_samples=n_samples, n_ctx=15, seed=42)
dsets = [dataset1,dataset2]
poly_dataset = PolyModelDataset(dsets, n_ctx=poly_hl_model.cfg.n_ctx)

test_samples = 100
input = poly_dataset.get_dataset()[:test_samples][0]
output = poly_hl_model((input, None, None)).cpu()
expected = poly_dataset.get_dataset()[:test_samples][1]
print(input)
print(torch.allclose(output, expected))

/Users/evananders/far_cluster/circuits-benchmark-saes/poly_bench/cases/duplicate_remover.py:161: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _, cache = hl_model.run_with_cache((t.tensor(sample).unsqueeze(0), None, None))


tensor([[1, 3, 0,  ..., 1, 0, 1],
        [1, 3, 0,  ..., 1, 0, 0],
        [0, 4, 0,  ..., 0, 0, 1],
        ...,
        [1, 3, 1,  ..., 0, 0, 1],
        [0, 4, 0,  ..., 0, 1, 0],
        [0, 4, 0,  ..., 2, 2, 2]])
True


/Users/evananders/far_cluster/circuits-benchmark-saes/poly_bench/poly_hl_model.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.cat([torch.tensor(dataset.labels) for dataset in datasets], dim=0)


: 

: 

: 

In [ ]:
ll_model = poly_hl_model.get_ll_model()
corr = poly_hl_model.get_correspondence()
for k, v in corr.items():
    print(k, v)

# n_epochs = 100
# training_args = {
#     "batch_size": 256,
#     "lr": 3e-4,
#     "num_workers": 0,
#     "use_single_loss": False,
#     "behavior_weight": 1., #basically doubles the strict weight's job.
#     "iit_weight": 1.,
#     "strict_weight": 0.4,
#     "clip_grad_norm": 1.0,
#     "iit_weight_schedule" : lambda s, i: s,
#     "strict_weight_schedule" : lambda s, i: s,
#     "behavior_weight_schedule" : lambda s, i: s, #0.955*s if 0.955**i > 0.01 else s, #have behavior weight decay over time
#     "early_stop" : True,
#     "lr_scheduler": torch.optim.lr_scheduler.LinearLR,
#     "scheduler_kwargs": dict(start_factor=1, end_factor=0, total_iters=n_epochs),
#     "scheduler_val_metric": ["val/accuracy", "val/IIA"], #for ReduceLRonPlateau
#     "scheduler_mode": "max", #for ReduceLRonPlateau
# }

# training_args['iit_weight'] = 1.
# training_args['strict_weight'] = 1.
# training_args['optimizer_kwargs']['betas'] = (0.8, 0.9)
# training_args['optimizer_kwargs']['weight_decay'] = 1e-4
train_set, test_set = poly_dataset.get_IIT_train_test_set()
model_pair = StrictIITModelPair(hl_model=poly_hl_model, ll_model=ll_model, corr=corr, training_args=training_args)
print(model_pair.nodes_not_in_circuit)
model_pair.train(
    train_set=train_set,
    test_set=test_set,
    optimizer_cls=torch.optim.AdamW,
    epochs=n_epochs,
    # optimizer_kwargs=dict(weight_decay=1e-4),
)

input_hook {LLNode(name='blocks.0.hook_resid_pre', index=[:], subspace=None)}
mlp_hooks.0 {LLNode(name='blocks.0.mlp.hook_post', index=[:], subspace=None)}
attn_hooks.0.0 {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 0, :], subspace=None)}
attn_hooks.0.1 {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 1, :], subspace=None)}
task_hook {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 2, :], subspace=None)}
mlp_hooks.1 {LLNode(name='blocks.1.mlp.hook_post', index=[:], subspace=None)}
[LLNode(name='blocks.0.attn.hook_z', index=[:, :, 3, :], subspace=None), LLNode(name='blocks.1.attn.hook_z', index=[:, :, 0, :], subspace=None), LLNode(name='blocks.1.attn.hook_z', index=[:, :, 1, :], subspace=None), LLNode(name='blocks.1.attn.hook_z', index=[:, :, 2, :], subspace=None), LLNode(name='blocks.1.attn.hook_z', index=[:, :, 3, :], subspace=None)]
training_args={'batch_size': 256, 'num_workers': 0, 'early_stop': True, 'lr_scheduler': <class 'torch.optim.lr_scheduler.LinearLR'>, 'scheduler_va

Epoch 1: lr: 9.99e-04, iit_weight: 1.00e+00, behavior_weight: 5.00e-01, strict_weight: 5.00e-01, train/iit_loss: 1.28e+00, train/behavior_loss: 5.97e-01, train/strict_loss: 3.14e-01, val/iit_loss: 1.01e+00, val/IIA: 58.38, val/accuracy: 69.71, val/strict_accuracy: 68.23
Epoch 2: lr: 9.98e-04, iit_weight: 1.00e+00, behavior_weight: 5.00e-01, strict_weight: 5.00e-01, train/iit_loss: 8.94e-01, train/behavior_loss: 3.31e-01, train/strict_loss: 1.94e-01, val/iit_loss: 8.65e-01, val/IIA: 63.64, val/accuracy: 83.48, val/strict_accuracy: 80.62
Epoch 3: lr: 9.96e-04, iit_weight: 1.00e+00, behavior_weight: 5.00e-01, strict_weight: 5.00e-01, train/iit_loss: 5.51e-01, train/behavior_loss: 2.04e-01, train/strict_loss: 1.53e-01, val/iit_loss: 7.91e-01, val/IIA: 70.08, val/accuracy: 87.35, val/strict_accuracy: 84.66
Epoch 4: lr: 9.95e-04, iit_weight: 1.00e+00, behavior_weight: 5.00e-01, strict_weight: 5.00e-01, train/iit_loss: 7.25e-01, train/behavior_loss: 1.51e-01, train/strict_loss: 1.29e-01, val/

: 

: 

: 

## Duplicate Remover + LeftGreater + ParensBalancer

In [ ]:
cases = [HighLevelDuplicateRemover, HighLevelLeftGreater, HighLevelParensBalanceChecker]
poly_hl_model = PolyHLModel(hl_classes=cases, size_expansion=2)
corr = poly_hl_model.get_correspondence()

dataset1 = DuplicateRemoverDataset(N_samples=n_samples, n_ctx=15, seed=42)
dataset2 = LeftGreaterDataset(N_samples=n_samples, n_ctx=15, seed=42)
dataset3 = BalancedParensDataset(N_samples=n_samples, n_ctx=15, seed=42)
dsets = [dataset1,dataset2, dataset3]
poly_dataset = PolyModelDataset(dsets, n_ctx=poly_hl_model.cfg.n_ctx)


: 

: 

: 

In [ ]:
ll_model = poly_hl_model.get_ll_model()
corr = poly_hl_model.get_correspondence()
for k, v in corr.items():
    print(k, v)

# n_epochs = 1000
# training_args = {
#     "batch_size": 256,
#     "lr": 1e-3,
#     "num_workers": 0,
#     "use_single_loss": True,
#     "behavior_weight": 0., #basically doubles the strict weight's job.
#     "iit_weight": 1.,
#     "strict_weight": 1.,
#     "clip_grad_norm": 1.0,
#     "iit_weight_schedule" : lambda s, i: s,
#     "strict_weight_schedule" : lambda s, i: s,
#     "behavior_weight_schedule" : lambda s, i: s, #0.955*s if 0.955**i > 0.01 else s, #have behavior weight decay over time
#     "early_stop" : True,
#     "lr_scheduler": torch.optim.lr_scheduler.LinearLR,
#     "scheduler_kwargs": dict(start_factor=1, end_factor=0, total_iters=n_epochs),
#     "scheduler_val_metric": ["val/accuracy", "val/IIA"], #for ReduceLRonPlateau
#     "scheduler_mode": "max", #for ReduceLRonPlateau
# }
train_set, test_set = poly_dataset.get_IIT_train_test_set()
model_pair = StrictIITModelPair(hl_model=poly_hl_model, ll_model=ll_model, corr=corr, training_args=training_args)
print(model_pair.nodes_not_in_circuit)
model_pair.train(
    train_set=train_set,
    test_set=test_set,
    optimizer_cls=torch.optim.AdamW,
    epochs=n_epochs,
    # optimizer_kwargs=dict(weight_decay=1e-4),
)

input_hook {LLNode(name='blocks.0.hook_resid_pre', index=[:], subspace=None)}
mlp_hooks.0 {LLNode(name='blocks.0.mlp.hook_post', index=[:], subspace=None)}
attn_hooks.0.0 {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 0, :], subspace=None)}
attn_hooks.0.1 {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 1, :], subspace=None)}
task_hook {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 2, :], subspace=None)}
attn_hooks.0.3 {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 3, :], subspace=None)}
mlp_hooks.1 {LLNode(name='blocks.1.mlp.hook_post', index=[:], subspace=None)}
mlp_hooks.2 {LLNode(name='blocks.2.mlp.hook_post', index=[:], subspace=None)}
attn_hooks.2.3 {LLNode(name='blocks.2.attn.hook_z', index=[:, :, 3, :], subspace=None)}
[LLNode(name='blocks.1.attn.hook_z', index=[:, :, 0, :], subspace=None), LLNode(name='blocks.1.attn.hook_z', index=[:, :, 1, :], subspace=None), LLNode(name='blocks.1.attn.hook_z', index=[:, :, 2, :], subspace=None), LLNode(name='blocks.1.attn.hook_z', 

Epoch 1: lr: 9.99e-04, iit_weight: 1.00e+00, behavior_weight: 5.00e-01, strict_weight: 5.00e-01, train/iit_loss: 1.30e+00, train/behavior_loss: 6.19e-01, train/strict_loss: 3.14e-01, val/iit_loss: 9.41e-01, val/IIA: 59.83, val/accuracy: 63.96, val/strict_accuracy: 63.69
Epoch 2: lr: 9.98e-04, iit_weight: 1.00e+00, behavior_weight: 5.00e-01, strict_weight: 5.00e-01, train/iit_loss: 8.16e-01, train/behavior_loss: 3.61e-01, train/strict_loss: 1.93e-01, val/iit_loss: 5.76e-01, val/IIA: 78.92, val/accuracy: 80.19, val/strict_accuracy: 79.14
Epoch 3: lr: 9.96e-04, iit_weight: 1.00e+00, behavior_weight: 5.00e-01, strict_weight: 5.00e-01, train/iit_loss: 7.98e-01, train/behavior_loss: 2.12e-01, train/strict_loss: 1.41e-01, val/iit_loss: 7.64e-01, val/IIA: 70.80, val/accuracy: 89.67, val/strict_accuracy: 89.09
Epoch 4: lr: 9.95e-04, iit_weight: 1.00e+00, behavior_weight: 5.00e-01, strict_weight: 5.00e-01, train/iit_loss: 6.30e-01, train/behavior_loss: 1.49e-01, train/strict_loss: 1.04e-01, val/

: 

: 

: 

## Duplicate Remover + LeftGreater + ParensBalancer + Unique Extractor

In [ ]:
cases = [HighLevelDuplicateRemover, HighLevelLeftGreater, HighLevelParensBalanceChecker, HighLevelUniqueExtractor]
poly_hl_model = PolyHLModel(hl_classes=cases, size_expansion=2)
corr = poly_hl_model.get_correspondence()

dataset1 = DuplicateRemoverDataset(N_samples=n_samples, n_ctx=15, seed=42)
dataset2 = LeftGreaterDataset(N_samples=n_samples, n_ctx=15, seed=42)
dataset3 = BalancedParensDataset(N_samples=n_samples, n_ctx=15, seed=42)
dataset4 = UniqueExtractorDataset(N_samples=n_samples, n_ctx=15, seed=42)
dsets = [dataset1,dataset2, dataset3, dataset4]
poly_dataset = PolyModelDataset(dsets, n_ctx=poly_hl_model.cfg.n_ctx)

/Users/evananders/far_cluster/circuits-benchmark-saes/poly_bench/poly_hl_model.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens = torch.cat([torch.tensor(dataset.tokens) for dataset in datasets], dim=0)
/Users/evananders/far_cluster/circuits-benchmark-saes/poly_bench/poly_hl_model.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.cat([torch.tensor(dataset.labels) for dataset in datasets], dim=0)


: 

: 

: 

In [ ]:
ll_model = poly_hl_model.get_ll_model()
corr = poly_hl_model.get_correspondence()
for k, v in corr.items():
    print(k, v)

train_set, test_set = poly_dataset.get_IIT_train_test_set()
model_pair = StrictIITModelPair(hl_model=poly_hl_model, ll_model=ll_model, corr=corr, training_args=training_args)
print(model_pair.nodes_not_in_circuit)
model_pair.train(
    train_set=train_set,
    test_set=test_set,
    optimizer_cls=torch.optim.AdamW,
    epochs=n_epochs,
)

input_hook {LLNode(name='blocks.0.hook_resid_pre', index=[:], subspace=None)}
output_hook {LLNode(name='blocks.3.mlp.hook_post', index=[:], subspace=None)}
mlp_hooks.0 {LLNode(name='blocks.0.mlp.hook_post', index=[:], subspace=None)}
attn_hooks.0.0 {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 0, :], subspace=None)}
attn_hooks.0.1 {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 1, :], subspace=None)}
attn_hooks.0.2 {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 2, :], subspace=None)}
attn_hooks.0.3 {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 3, :], subspace=None)}
mlp_hooks.1 {LLNode(name='blocks.1.mlp.hook_post', index=[:], subspace=None)}
task_hook {LLNode(name='blocks.1.attn.hook_z', index=[:, :, 0, :], subspace=None)}
mlp_hooks.2 {LLNode(name='blocks.2.mlp.hook_post', index=[:], subspace=None)}
attn_hooks.2.3 {LLNode(name='blocks.2.attn.hook_z', index=[:, :, 3, :], subspace=None)}
[LLNode(name='blocks.1.attn.hook_z', index=[:, :, 1, :], subspace=None), LLNode(name='b

Training Epochs:   0%|          | 0/300 [00:00<?, ?it/s]

Training Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch 1: lr: 9.97e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 4.00e-01, train/iit_loss: 1.10e+00, train/behavior_loss: 8.93e-01, train/strict_loss: 1.65e-01, val/iit_loss: 5.60e-01, val/IIA: 68.18, val/accuracy: 80.32, val/strict_accuracy: 78.35
Epoch 2: lr: 9.94e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 4.00e-01, train/iit_loss: 8.38e-01, train/behavior_loss: 3.98e-01, train/strict_loss: 1.03e-01, val/iit_loss: 3.77e-01, val/IIA: 73.65, val/accuracy: 88.12, val/strict_accuracy: 86.70
Epoch 3: lr: 9.91e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 4.00e-01, train/iit_loss: 5.70e-01, train/behavior_loss: 2.39e-01, train/strict_loss: 5.73e-02, val/iit_loss: 3.33e-01, val/IIA: 76.70, val/accuracy: 93.27, val/strict_accuracy: 91.51
Epoch 4: lr: 9.88e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 4.00e-01, train/iit_loss: 5.50e-01, train/behavior_loss: 2.07e-01, train/strict_loss: 4.97e-02, val/

KeyboardInterrupt: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 